In [1]:
import os
os.environ.pop("PYTORCH_CUDA_ALLOC_CONF", None)
# or: os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

import torch
print(torch.__version__, torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


1.13.1+cu117 11.7
CUDA available: True
NVIDIA A100 80GB PCIe


In [2]:
import sys, os

# Path of current working directory (where notebook is running)
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, "backdoors101"))

In [3]:
from backdoors101.training import run

In [4]:
# 1) imports you need (same modules the script uses)
import yaml
from types import SimpleNamespace
from datetime import datetime
from helper import Helper
# logging optional
import logging
logger = logging.getLogger('logger')

In [5]:

# 2) set args (like argparse would)
params_path = "./config_loss.yml"   # change to the config you want
exp_name = "mnist_loss_test_run"
commit = "none"   # or get_current_git_hash() if available/importable


In [6]:
# 3) load params and attach fields the script expects
with open(params_path, "r") as f:
    params = yaml.load(f, Loader=yaml.FullLoader)


In [7]:
params['current_time'] = datetime.now().strftime('%b.%d_%H.%M')
params['commit'] = commit
params['name'] = exp_name

### Load MNIST Dataset

In [8]:
# # 1) Point torchvision's MNIST to a reliable mirror (S3) and keep the same MD5s
# from torchvision.datasets import mnist as tv_mnist

# tv_mnist.MNIST.resources = [
#     ("https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz",
#      "f68b3c2dcbeaaa9fbdd348bbdeb94873"),
#     ("https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz",
#      "d53e105ee54ea40749a09fcbcd1e9432"),
#     ("https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz",
#      "9fb629c4189551a2d022fa330f9573f3"),
#     ("https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz",
#      "ec29112dd5afa0611ce80d1b7f02629c"),
# ]

# # 2) Make sure your params point to a writable path
# params['data_path'] = params.get('data_path', 'data')

In [9]:
# 4) create Helper (this sets up datasets, model, attack, etc.)
helper = Helper(params)

2025-09-30 23:34:58 - WARNING  - Logging to: saved_models/model_MNIST_Sep.30_23.34_mnist_loss_test_run
2025-09-30 23:34:58 - ERROR    - LINK: <a href="https://github.com/ebagdasa/backdoors/tree/none">https://github.com/ebagdasa/backdoors/tree/none</a>


### Creating the Backdoor Model

In [10]:
# OPTIONAL: shorten the run for testing
# run a fast smoke-test by limiting batches / epochs
helper.params.epochs = 5                  # 20 epochs (for quick test)
helper.params.max_batch_id = 50           # limit batches per epoch (small)
# If fl mode is enabled in config, set it off for single-node run:
helper.params.fl = False

### Running the model

In [11]:
# 5) run: choose run(helper) for centralized training, or fl_run(helper) for federated mode
# Import the functions defined in the script file (if you pasted them into a cell), otherwise
# re-define 'run' and 'fl_run' or import them from training module if training.py exposes them.
# If you copied the script code into the notebook, you should have run(), test(), train(), fl_run() available.
run(helper)   # runs full train/test loop (with the small epoch/batch limits above)
# or for federated:
# fl_run(helper)


100it [00:02, 38.65it/s]
2025-09-30 23:35:01 - WARNING  - Backdoor False. Epoch:     0. Accuracy: Top-1: 8.61 | Loss: value: 2.30
0it [00:00, ?it/s]/home/hice1/vchopra37/scratch/projects/ml_security/paper_presentation/paper_presentation/lib/python3.8/site-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
2025-09-30 23:35:01 - INFO     - Epoch:   1. Batch:     0/938.  Losses: ['backdoor: 2.48', 'normal: 2.30', 'total: 2.30']. Scales: ['backdoor: 0.00', 'normal: 1.00']
50it [00:00, 52.46it/s]
100it [00:01, 62.02it/s]
2025-09-30 23:35:04 - WARNING  - Backdoor False. Epoch:     1. Accuracy: Top-1: 63.38 | Loss: value: 0.89
100it [00:01, 60.09it/s]
2025-09-30 23:35:06 - WARNING  - Backdoor True . Epoch:     1. Accuracy: Top-1: 68.18 | Loss: value: 0.79
2025-09-30 23:35:06 - INFO     - Saving model to saved_models/model_MNIST_Sep.30_23.3

### Testing the Model